In [11]:
# !pip install qiskit==0.43.3

In [12]:
import numpy as np
from qiskit.opflow import PauliOp, I, Z, Plus, X, Minus
from qiskit.providers.aer import QasmSimulator
from qiskit.utils import QuantumInstance
import networkx as nx


In [ ]:
"""Minimal mining solution with Adiabatic State Preparation (ASP)."""


def solve_asp(
    mine_config,
    evol_time=10,
    nsteps=20,
    quantum_instance=None
):
    """
    Solve the open-pit mining problem using Adiabatic State Preparation (ASP).
    Returns only the optimal configuration and profit.
    
    Parameters
    ----------
    mine_config : numpy.ndarray
        A 2D numpy array representing the mine configuration.
    evol_time : float
        Time for the adiabatic evolution.
    nsteps : int
        Number of discrete time blocks for Trotterization.
    quantum_instance : QuantumInstance, optional
        Quantum instance to run the algorithm.
        
    Returns
    -------
    tuple
        (optimal_config, profit) tuple where:
        - optimal_config: The optimal configuration as a bitstring.
        - profit: The profit of the optimal configuration.
    """
    # Initialize quantum instance if not provided
    if quantum_instance is None:
        quantum_instance = QuantumInstance(QasmSimulator(method="statevector"))
    
    # Process the mine data
    dat = np.array(mine_config, dtype=float)
    rows, cols = dat.shape
    graph = {}  # parent to child relationships
    idx2cord = []
    cord2idx = {}
    
    # Initialize graph and mapping
    for r in range(rows):
        for c in range(cols):
            # Set invalid cells to infinity
            if c < r or c > cols - r - 1:
                dat[r, c] = float("inf")
            
            # Process valid cells
            if dat[r, c] < float("inf"):
                idx2cord.append((r, c))
                cord2idx[(r, c)] = len(idx2cord) - 1
                idx = cord2idx[(r, c)]
                graph[idx] = []
                
                # Connect to parent nodes
                for pr, pc in [(r - 1, c - 1), (r - 1, c), (r - 1, c + 1)]:
                    if (0 <= pr < rows and 
                        0 <= pc < cols and 
                        dat[pr, pc] < float("inf")):
                        parent_idx = cord2idx.get((pr, pc))
                        if parent_idx is not None:
                            graph[parent_idx].append(idx)
    
    nqubits = len(idx2cord)
    
    # Generate Hamiltonians
    # Smoothness Hamiltonian
    Hs = 0 * I ^ nqubits
    for i in range(nqubits):
        for j in graph.get(i, []):
            Hs += (
                (I ^ nqubits) - ((I ^ (nqubits - i - 1)) ^ Z ^ (I ^ i))
            ) @ ((I ^ nqubits) + ((I ^ (nqubits - j - 1)) ^ Z ^ (I ^ j)))
    Hs = 0.25 * Hs
    
    # Profit Hamiltonian
    Hp = 0 * I ^ nqubits
    for i in range(nqubits):
        Hp += float(dat[idx2cord[i]]) * (
            (I ^ nqubits) - ((I ^ (nqubits - i - 1)) ^ Z ^ (I ^ i))
        )
    Hp = 0.5 * Hp
    
    # Calculate penalty
    penalty = float(
        np.linalg.norm(np.where(dat.flat != np.inf), ord=2) / nqubits
    ) * 3.8
    
    # Generate the final Hamiltonian
    H = (-Hp + penalty * Hs).reduce()
    
    # Create ASP algorithm components
    initial_state = (Minus ^ nqubits).to_circuit()
    initial_operator = X ^ (I ^ (nqubits - 1))
    for i in range(1, nqubits):
        initial_operator += (I ^ i) ^ X ^ (I ^ (nqubits - i - 1))
    
    # Build the circuit
    circuit = initial_state.copy()
    for i in range(nsteps):
        xi = (0.5 + i) / nsteps
        circuit.hamiltonian(
            H,
            xi * evol_time / nsteps,
            list(range(nqubits)),
        )
        circuit.hamiltonian(
            initial_operator,
            (1 - xi) * evol_time / nsteps,
            list(range(nqubits)),
        )
    
    # Execute the circuit
    if quantum_instance.is_statevector:
        eigenstate = quantum_instance.execute(circuit).get_statevector(circuit)
        if isinstance(eigenstate, np.ndarray):
            idx = np.argmax(eigenstate * eigenstate.conj())
            optimal_config = format(idx, f"0{nqubits}b")[-nqubits:]
        else:
            return None, 0
    else:
        result = quantum_instance.execute(circuit.measure_all(inplace=False))
        counts = result.get_counts()
        optimal_config, _ = max(counts.items(), key=lambda x: x[1])
        # Convert to LSB-first format if needed
        if len(optimal_config) > nqubits:
            optimal_config = optimal_config[-nqubits:]
    
    # Calculate profit for optimal configuration
    bitstring = "".join(list(optimal_config)[::-1])  # Convert to LSB-first format
    profit = sum([
        dat[idx2cord[i]]
        for i in range(nqubits)
        if i < len(bitstring) and bitstring[i] == "1"
    ])
    
    return optimal_config, profit

# Define a mine configuration
mine_data = np.array([
    [-2.0,                  3.0, -1.0,        -2.0,        -1.0],
    [float('inf'),          1.0, -5.0,        10.0, float('inf')],
    [float('inf'), float('inf'), 4.0, float('inf'), float('inf')]
])

# Solve the mining problem
optimal_config, profit = solve_asp(
    mine_data,
    evol_time=10,
    nsteps=20
)

print(f"Optimal configuration: {optimal_config}")
print(f"Profit: {profit}")

/var/folders/tt/3h_jlt8571z664b95jr80v_40000gn/T/ipykernel_11149/1897050683.py:34: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  quantum_instance = QuantumInstance(QasmSimulator(method="statevector"))


Optimal configuration: 110100000
Profit: 15.0


In [15]:


def solve_mining_classical(mine_config):
    """
    Solve the open-pit mining problem using the maximum flow/minimum cut approach.
    
    Parameters:
    mine_config: 2D numpy array with profit/cost values for each block
    
    Returns:
    optimal_config: String representing which blocks to mine
    profit: Total profit of the optimal configuration
    """
    # Process the mine data
    dat = np.array(mine_config, dtype=float)
    rows, cols = dat.shape
    
    # Create mappings between coordinates and indices
    idx2cord = []
    cord2idx = {}
    
    for r in range(rows):
        for c in range(cols):
            # Set invalid cells to infinity
            if c < r or c > cols - r - 1:
                dat[r, c] = float("inf")
            
            # Process valid cells
            if dat[r, c] < float("inf"):
                idx2cord.append((r, c))
                cord2idx[(r, c)] = len(idx2cord) - 1
    
    nblocks = len(idx2cord)
    
    # Create directed graph
    G = nx.DiGraph()
    
    # Add nodes for each block
    G.add_nodes_from(range(nblocks))
    
    # Add source and sink nodes
    source = nblocks
    sink = nblocks + 1
    G.add_node(source)
    G.add_node(sink)
    
    # Add edges for precedence constraints
    for r in range(rows):
        for c in range(cols):
            if (r, c) in cord2idx:
                block_idx = cord2idx[(r, c)]
                
                # Add edges to children (blocks below this one)
                for child_r, child_c in [(r+1, c-1), (r+1, c), (r+1, c+1)]:
                    if (0 <= child_r < rows and 
                        0 <= child_c < cols and 
                        (child_r, child_c) in cord2idx):
                        child_idx = cord2idx[(child_r, child_c)]
                        G.add_edge(block_idx, child_idx, capacity=float('inf'))
    
    # Add edges from source to positive blocks and from negative blocks to sink
    for i in range(nblocks):
        r, c = idx2cord[i]
        value = dat[r, c]
        if value >= 0:
            G.add_edge(source, i, capacity=value)
        else:
            G.add_edge(i, sink, capacity=-value)
    
    # Find maximum flow / minimum cut
    _, flow_dict = nx.maximum_flow(G, source, sink)
    
    # Determine which blocks to mine based on the minimum cut
    reachable = nx.descendants(G, source)
    reachable.add(source)
    
    # Create optimal configuration
    config = ''
    for i in range(nblocks):
        if i in reachable:
            config += '1'  # Mine this block
        else:
            config += '0'  # Don't mine this block
    
    # Calculate total profit
    profit = sum(dat[idx2cord[i]] for i in range(nblocks) if config[i] == '1')
    
    return config, profit

optimal_config, profit = solve_mining_classical(
    mine_data
)

print(f"Optimal configuration: {optimal_config}")
print(f"Profit: {profit}")

Optimal configuration: 010001111
Profit: 13.0


In [16]:
import numpy as np
from itertools import product

def solve_mining_brute_force(mine_config):
    """
    Solve the open-pit mining problem using brute force approach.
    
    This function tries all possible configurations of mining blocks
    and returns the one that maximizes profit while respecting the
    mining precedence constraints.
    
    Parameters:
    -----------
    mine_config : np.ndarray
        2D array representing the mine with profit/cost values for each block
        
    Returns:
    --------
    tuple
        (optimal_config, profit) where:
        - optimal_config: String of 0s and 1s representing which blocks to mine
        - profit: The total profit of the optimal configuration
    """
    # Process the mine data
    dat = np.array(mine_config, dtype=float)
    rows, cols = dat.shape
    
    # Create mappings between coordinates and indices
    idx2cord = []
    cord2idx = {}
    
    # Graph to store parent-child relationships
    parents = {}  # For each block, store its parents (blocks above it)
    
    # Initialize valid blocks and their relationships
    for r in range(rows):
        for c in range(cols):
            # Set invalid cells to infinity
            if c < r or c > cols - r - 1:
                dat[r, c] = float("inf")
            
            # Process valid cells
            if dat[r, c] < float("inf"):
                idx2cord.append((r, c))
                current_idx = len(idx2cord) - 1
                cord2idx[(r, c)] = current_idx
                
                # Initialize parents list for this block
                parents[current_idx] = []
                
                # Find parents (blocks above this one)
                for pr, pc in [(r-1, c-1), (r-1, c), (r-1, c+1)]:
                    if (0 <= pr < rows and 
                        0 <= pc < cols and 
                        (pr, pc) in cord2idx):
                        parents[current_idx].append(cord2idx[(pr, pc)])
    
    nblocks = len(idx2cord)
    print(f"Number of valid blocks: {nblocks}")
    print(f"Total configurations to check: {2**nblocks}")
    
    # Brute force approach: try all possible configurations
    best_config = None
    best_profit = float('-inf')
    
    # Generate all possible binary configurations
    for config_tuple in product([0, 1], repeat=nblocks):
        config = ''.join(map(str, config_tuple))
        
        # Check if configuration is valid (respects precedence constraints)
        valid = True
        for i in range(nblocks):
            if config[i] == '1':  # If we're mining this block
                # All parent blocks must also be mined
                for parent in parents[i]:
                    if config[parent] == '0':
                        valid = False
                        break
            if not valid:
                break
        
        if valid:
            # Calculate profit for this configuration
            profit = sum(dat[idx2cord[i]] for i in range(nblocks) if config[i] == '1')
            
            # Update best configuration if this is better
            if profit > best_profit:
                best_profit = profit
                best_config = config
    
    # If we couldn't find a valid configuration (shouldn't happen with typical mines)
    if best_config is None:
        return "0" * nblocks, 0.0
    
    return best_config, best_profit

def print_mine_with_solution(mine_config, solution):
    """
    Visualize the mine configuration with the solution overlaid.
    
    Parameters:
    -----------
    mine_config : np.ndarray
        2D array representing the mine
    solution : str
        String of 0s and 1s representing which blocks to mine
    """
    dat = np.array(mine_config, dtype=float)
    rows, cols = dat.shape
    
    # Create mappings between coordinates and indices
    idx2cord = []
    cord2idx = {}
    
    for r in range(rows):
        for c in range(cols):
            if c < r or c > cols - r - 1:
                dat[r, c] = float("inf")
            if dat[r, c] < float("inf"):
                idx2cord.append((r, c))
                cord2idx[(r, c)] = len(idx2cord) - 1
    
    # Create a visualization matrix
    vis_matrix = np.full((rows, cols), ' ', dtype=object)
    
    for r in range(rows):
        for c in range(cols):
            if (r, c) in cord2idx:
                idx = cord2idx[(r, c)]
                if idx < len(solution):
                    if solution[idx] == '1':
                        vis_matrix[r, c] = 'M'  # Mine this block
                    else:
                        vis_matrix[r, c] = '.'  # Don't mine this block
                    
                    # Also show the profit/cost value
                    vis_matrix[r, c] = f"{vis_matrix[r, c]}{dat[r, c]:>5.1f}"
                else:
                    vis_matrix[r, c] = 'X'  # This shouldn't happen
            else:
                vis_matrix[r, c] = '  inf  '  # Inaccessible block
    
    # Print the visualization
    print("\nMine Configuration with Solution:")
    print("-" * (cols * 8))
    for r in range(rows):
        print("|", end="")
        for c in range(cols):
            print(f" {vis_matrix[r, c]} |", end="")
        print("\n" + "-" * (cols * 8))
    
    # Print legend
    print("\nLegend:")
    print("M - Mine this block")
    print(". - Don't mine this block")
    print("inf - Inaccessible block")


optimal_config, profit = solve_mining_brute_force(mine_data)

print(f"\nOptimal configuration: {optimal_config}")
print(f"Total profit: {profit}")

# Visualize the solution
print_mine_with_solution(mine_data, optimal_config)

Number of valid blocks: 9
Total configurations to check: 512

Optimal configuration: 011110010
Total profit: 9.0

Mine Configuration with Solution:
----------------------------------------
| . -2.0 | M  3.0 | M -1.0 | M -2.0 | M -1.0 |
----------------------------------------
|   inf   | .  1.0 | . -5.0 | M 10.0 |   inf   |
----------------------------------------
|   inf   |   inf   | .  4.0 |   inf   |   inf   |
----------------------------------------

Legend:
M - Mine this block
. - Don't mine this block
inf - Inaccessible block
